# 99_fig9_taylor_vs_modtaylor

Figure 9: ergodic distributions under Taylor vs modified Taylor (normal/bad regimes).

In [ ]:
import os, sys, json, numpy as np, torch, matplotlib.pyplot as plt
import pathlib

def _find_project_root():
    here = pathlib.Path.cwd().resolve()
    for p in [here, *here.parents]:
        if (p / "src").is_dir():
            return p
    # Common Google Colab clone location
    cand = pathlib.Path("/content/econml")
    if (cand / "src").is_dir():
        return cand
    raise RuntimeError("Could not find project root containing src/. If on Colab, clone repo to /content/econml.")

PROJECT_ROOT = _find_project_root()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.config import ModelParams
from src.io_utils import load_json, load_npz, load_torch, load_selected_run, find_latest_run_dir
from src.deqn import PolicyNetwork

ART = str(PROJECT_ROOT / "artifacts" / "runs")

def get_run(policy: str) -> str:
    rd = load_selected_run(ART, policy)
    if rd is None:
        rd = find_latest_run_dir(ART, policy)
    if rd is None:
        raise RuntimeError(f"No runs found for policy={policy} under {ART}")
    return rd

def _parse_dtype(s: str):
    if s is None:
        return torch.float32
    if isinstance(s, torch.dtype):
        return s
    s = str(s)
    if "float64" in s:
        return torch.float64
    if "float32" in s:
        return torch.float32
    if "bfloat16" in s:
        return torch.bfloat16
    return torch.float32

def load_params_from_run(run_dir: str, *, device="cpu"):
    cfg = load_json(os.path.join(run_dir, "config.json"))
    p = cfg.get("params", {})
    dtype = _parse_dtype(p.get("dtype"))
    dev = device if device is not None else p.get("device","cpu")
    keep = {k:v for k,v in p.items() if k in ModelParams.__dataclass_fields__}
    keep["device"] = dev
    keep["dtype"] = dtype
    return ModelParams(**keep).to_torch()

def load_net_from_run(run_dir: str, d_in: int, d_out: int):
    cfg = load_json(os.path.join(run_dir, "config.json"))
    tc = cfg.get("train_cfg", {})
    hidden = tuple(tc.get("hidden_layers", (512,512)))
    activation = tc.get("activation", "selu")
    p = cfg.get("params", {})
    net_dtype = _parse_dtype(p.get("dtype"))
    net = PolicyNetwork(d_in, d_out, hidden=hidden, activation=activation).to(device="cpu", dtype=net_dtype)
    state = load_torch(os.path.join(run_dir, "weights.pt"), map_location="cpu")
    # state is usually a plain state_dict
    if isinstance(state, dict) and "state_dict" in state:
        state = state["state_dict"]
    net.load_state_dict(state)
    net.eval()
    return net

# --- paper reporting helpers ---
ann = lambda x: 400.0*x  # annualized percent (quarterly -> annual)


In [ ]:
from src.steady_states import solve_efficient_sss
from src.metrics import output_gap_from_consumption

run_t = get_run("taylor")
run_m = get_run("mod_taylor")

params_t = load_params_from_run(run_t)
params_m = load_params_from_run(run_m)

sim_t = load_npz(os.path.join(run_t, "sim_paths.npz"))
sim_m = load_npz(os.path.join(run_m, "sim_paths.npz"))

# Figure 9 compares policies under the same calibration.
for k in ModelParams.__dataclass_fields__.keys():
    if k in ("device", "dtype"):
        continue
    if abs(float(getattr(params_t, k)) - float(getattr(params_m, k))) > 1e-12:
        raise RuntimeError(f"Figure 9 requires identical calibration for Taylor and modified Taylor. Mismatch in {k}.")

params = params_t
eff = solve_efficient_sss(params)


def compute_series(sim):
    s = sim["s"].reshape(-1).astype(np.int64)
    pi = sim["pi"].reshape(-1)
    i = sim["i"].reshape(-1)
    x = output_gap_from_consumption(sim, eff, params=params, time_varying=True)
    r = ((1.0 + i[:-1]) / (1.0 + pi[1:])) - 1.0
    s_r = s[:-1]
    return {
        "s": s,
        "pi": 400.0 * pi,
        "x": 100.0 * x,
        "r": 400.0 * r,
        "s_r": s_r,
    }

ser_t = compute_series(sim_t)
ser_m = compute_series(sim_m)

fig, ax = plt.subplots(1, 3, figsize=(15, 4))

ax[0].hist(ser_t["pi"][ser_t["s"] == 0], bins=60, alpha=0.45, label="Taylor normal", color="tab:blue")
ax[0].hist(ser_t["pi"][ser_t["s"] == 1], bins=60, alpha=0.45, label="Taylor bad", color="tab:orange")
ax[0].hist(ser_m["pi"][ser_m["s"] == 0], bins=60, alpha=0.45, label="Mod Taylor normal", color="tab:green")
ax[0].hist(ser_m["pi"][ser_m["s"] == 1], bins=60, alpha=0.45, label="Mod Taylor bad", color="tab:red")
ax[0].set_title("Figure 9a: Inflation distribution")
ax[0].set_xlabel("Annualized percent")
ax[0].legend(fontsize=8)

ax[1].hist(ser_t["x"][ser_t["s"] == 0], bins=60, alpha=0.45, label="Taylor normal", color="tab:blue")
ax[1].hist(ser_t["x"][ser_t["s"] == 1], bins=60, alpha=0.45, label="Taylor bad", color="tab:orange")
ax[1].hist(ser_m["x"][ser_m["s"] == 0], bins=60, alpha=0.45, label="Mod Taylor normal", color="tab:green")
ax[1].hist(ser_m["x"][ser_m["s"] == 1], bins=60, alpha=0.45, label="Mod Taylor bad", color="tab:red")
ax[1].set_title("Figure 9b: Output gap distribution")
ax[1].set_xlabel("Percent")
ax[1].legend(fontsize=8)

ax[2].hist(ser_t["r"][ser_t["s_r"] == 0], bins=60, alpha=0.45, label="Taylor normal", color="tab:blue")
ax[2].hist(ser_t["r"][ser_t["s_r"] == 1], bins=60, alpha=0.45, label="Taylor bad", color="tab:orange")
ax[2].hist(ser_m["r"][ser_m["s_r"] == 0], bins=60, alpha=0.45, label="Mod Taylor normal", color="tab:green")
ax[2].hist(ser_m["r"][ser_m["s_r"] == 1], bins=60, alpha=0.45, label="Mod Taylor bad", color="tab:red")
ax[2].set_title("Figure 9c: Real rate distribution")
ax[2].set_xlabel("Annualized percent")
ax[2].legend(fontsize=8)

plt.tight_layout()
plt.show()
